In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle
from music21 import *
from glob import glob
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout
from keras.utils import np_utils

%run lstm_model.ipynb
np.set_printoptions(threshold=1000000)
bach_chorale_path = 'datasets/bach2'
parts_index = [0,1,2,3]
#parts_index = [0,1]
note_type = set()
"""
note_type:
0-127 : note pitch
128: slur-1st-part
129: slur-2nd-part
130: slur-3rd-part
131: slur-4th-part
132: rest
133: start symbol
134: end symbol
"""

def filter_files(files):
    file_list = []
    file_count = 0
    for i, file in enumerate(files, 1):
        s = converter.parse(file)
        if len(s.parts) == len(parts_index):
            print(i, file)
            file_count += 1
            file_list.append(file)
    return file_list, file_count

def chorale_to_inputs(s,i):
    inputs = []
    for j in range(16):
        inputs.append([133])
    for n in s.parts[i].flat.notes:
#         if n.isNote:
#             print(n.nameWithOctave, int(4 * n.quarterLength), n.offset)
#         else:
#             print(n.pitchNames, int(4 * n.quarterLength), n.offset)
        #print(n.expressions)
        if n.isNote:
            inputs.append([n.pitch.midi])
            note_type.add(frozenset([n.pitch.midi]))
        elif n.isChord:
            inputs.append([_note.pitch.midi for _note in n])
            note_type.add(frozenset([_note.pitch.midi for _note in n]))
        l = n.quarterLength
#         if l == 0:
#             l = 1
        for j in range(1, int(4 * l)):
            if i == 0:
                inputs.append([128])
                note_type.add(frozenset([128]))
            elif i == 1:
                inputs.append([129])
                note_type.add(frozenset([129]))
            elif i == 2:
                inputs.append([130])
                note_type.add(frozenset([130]))
            elif i == 3:
                inputs.append([131])
                note_type.add(frozenset([131]))
    for j in range(16):
        inputs.append([134])
    return inputs

def make_time_metadata(file):
    chorale = converter.parse(file)
    chorale_length = int(chorale.parts[0].duration.quarterLength * 4)
    time_metadata = np.zeros((chorale_length, ))
    
    for i in range(0, chorale_length):
        time_metadata[i] = i + 1
    return np.array(time_metadata, dtype=np.int32)

def append_notes(max_length, part_i_length, part_i_input):
    note_type.add(frozenset([132]))# 132-->note.Rest
    for num_appended_notes in range(max_length - part_i_length):
        part_i_input.append([132])
    return part_i_input

def check_parts_length(parts_length, inputs):
    max_length = max(parts_length)
    for i in parts_index:
        if parts_length[i] < max_length:
            inputs[i] = append_notes(max_length, parts_length[i], inputs[i])
                
def make_dataset(file_list):
    X = []
    time_metadata = []
    for file in tqdm(file_list):
        s = converter.parse(file)
        
        inputs = []
        parts_length = []
        time_metadata.append(make_time_metadata(file))
        for i in parts_index:
            part_input = chorale_to_inputs(s,i)
            parts_length.append(len(part_input))
            inputs.append(part_input)
        check_parts_length(parts_length, inputs)
        np_inputs = np.array(inputs)
        X.append(np_inputs)
    #X.reshape(X.shape[1:])
    #print(time_metadata)
    return X
def generator_X(notes):
    t = 0
    while True:
        X = []
        if t < notes.shape[0] - 16:
            X.append(notes[t:t+16])
        else: 
            t = 0
            continue
        t += 16
        X = np.array(X)
        yield (X)
def generate_notes(lstm_model, X):
    output_indices = []
    for i in range(len(parts_index)):
        model = load_model("models/bach"+str(i)+".h5")
#     model = load_model(lstm_model.name+".h5")
        gen = ((X) for (X) in generator_X(lstm_model.X))
#     predict_list = model.predict(X, batch_size=32)
        predict_list = model.predict_generator(gen, steps=300)
        part_notes = []
        for time in range(predict_list.shape[0]):
            for note_index in range(predict_list.shape[1]):
                part_notes.append(np.argmax(predict_list[time][note_index]))
        output_indices.append(part_notes)
    output_indices = np.array(output_indices)
    output_chorale = index_note_transform(output_indices, "to_note", len(parts_index))
    #output_indices = np.transpose(output_indices)
   # output_chorale = np.transpose(output_chorale)
    midi = chorale_to_midi(output_chorale)
    print(output_chorale)
    return midi

# def insert_note_chord(n, d, new_note, part):
    
# def notes_to_midi(n, d, new_note, part):
#     if (len(n) < 2): # note or rest
#         if n[0] == 132:
#             part.append(note.Rest())
#         elif n[0] < 128:
#             if d > 0:
#                 new_note.duration = duration.Duration(d / 4)
#                 part.append(new_note)
#                 d = 1
#                 new_note = note.Note(n[0])
#             else:
#                 d += 1
#     else: # chord
#         if d > 0:
#             new_note.duration = duration.Duration(d / 4)
#             part.append(new_note)
#             d = 1
#             new_note = chord.Chord(n)
#         else:
#             d += 1
def chorale_to_midi(chorale):
    score = stream.Score()
    for i, chorale_part in enumerate(chorale):
        part = stream.Part(id=i)
        d = 0
        new_note = note.Rest()
        for sound in chorale_part:
            sound = [n for n in sound]
            if (len(sound) < 2): # note or rest
                if sound[0] == 132:
                    part.append(note.Rest())
                elif sound[0] < 128:
                    if d > 0:
                        new_note.duration = duration.Duration(d / 4)
                        part.append(new_note)
                    d = 1
                    new_note = note.Note(sound[0])
                else:
                    d += 1
            else: # chord
                if d > 0:
                    new_note.duration = duration.Duration(d / 4)
                    part.append(new_note)
                d = 1
                new_note = chord.Chord(sound)
            
        new_note.duration = duration.Duration(d / 4)
        part.append(new_note)
        score.insert(part)
    return score
def make_note_type_list():
    note_type_list = []
    for i in note_type:
        note_type_list.append(i)
    return sorted(note_type_list)
# def transform_note_to_index(notes):
#     for time in range(notes.shape[0]):
#         for p_index in range(4):
#             notes[time][p_index] = note_dict[notes[time][p_index]]
#     return notes
def index_note_transform(inputs, option, pn):
    transformed_inputs = []
    if option == "to_note":
        note_dict = dict((index, note) for index, note in enumerate(note_type))
    elif option == "to_index":
        note_dict = dict((note, index) for index, note in enumerate(note_type))
        
    for p_index in range(pn):
        transformed_part = []
        for n in range(inputs.shape[1]):
            for keys in note_dict:
                if option == "to_note":
                    if inputs[p_index][n] == keys:
                        transformed_part.append(note_dict[keys])
                elif option == "to_index":
                    if frozenset(inputs[p_index][n]) == keys:
                        transformed_part.append(note_dict[keys])
        transformed_inputs.append(transformed_part)
    return np.array(transformed_inputs)
def notes_to_piano_roll(dataset_notes):
#     start_symbol = np.full((16, 1),133)
#     end_symbol = np.full((16, 1),134)
    note_type.add(frozenset([133]))
    note_type.add(frozenset([134]))
    for chorale_index in range(len(dataset_notes)):
        chorale = np.array(dataset_notes[chorale_index])
#         for i in range(len(parts_index)):
#             piano_roll_notes.append(np.concatenate((start_symbol, chorale[i], end_symbol)))
        chorale = index_note_transform(chorale, "to_index", len(parts_index))
        piano_roll_notes = np.transpose(chorale)
        #piano_roll_notes = index_note_transform(transposed_chorale, "to_index", 4)
        yield piano_roll_notes
def notes_to_onehot(notes):
    note_dict = note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = []
        for note in n:
            a.append(np.array(note == np.arange(0, len(note_dict)), dtype=np.int32))
        onehot.append(a)
    return np.array(onehot)
def part_notes_to_onehot(notes):
    note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = np.zeros(len(note_dict,),dtype=np.int32)
        for note in n:
            a[note] = 1
        onehot.append(a)
    return np.array(onehot)

def load_files():
    if os.path.exists(bach_chorale_path):
#         file_list = filter_files(glob(bach_chorale_path))
        file_list, num_file = filter_files(glob(bach_chorale_path + '/*.mxl') +
                                 glob(bach_chorale_path + '/*.mid'))
        print(num_file)
        dataset_notes = make_dataset(file_list)
        shuffle(dataset_notes)
        chorale_list = [notes for chorale_index, notes in enumerate(notes_to_piano_roll(dataset_notes))]
        notes = chorale_list[0]
        for i, n in enumerate(chorale_list):
            if i != 0:
                notes = np.concatenate((notes, n))
        print(notes.shape)
        Y_onehot_notes = notes_to_onehot(notes)
        X_onehot_notes = part_notes_to_onehot(notes)
#         notes = notes[:,:,np.newaxis]
        for i in range(len(parts_index)):
            lstm_model = LSTM_Model(X=X_onehot_notes,Y=Y_onehot_notes,epochs=10, output_dim=len(note_type), name="bach"+str(i))
            gen = ((X,Y) for (X, Y) in lstm_model.generator(X_onehot_notes, Y_onehot_notes, i))
            lstm_model.model2()
            lstm_model.train_model(gen)

        score = generate_notes(lstm_model, notes)
                
        output_file = str(lstm_model.epochs) + '-epochs_' + str(num_file) + '-samples.mid'
        mf = midi.translate.music21ObjectToMidiFile(score)
        mf.open(output_file, 'wb')
        mf.write()
        mf.close()
        print("File " + output_file + " written")
    else:
        print("The file path is wrong!!")
    
def main():
    load_files()
if __name__ == "__main__":
    main()

1 datasets/bach2\bwv292.mxl
2 datasets/bach2\bwv293.mxl
3 datasets/bach2\bwv294.mxl
3


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 12.47it/s]


(768, 4)
Another model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 16, 37)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 16, 200)           190400    
_________________________________________________________________
dropout_11 (Dropout)         (None, 16, 200)           0         
_________________________________________________________________
dense_21 (Dense)             (None, 16, 200)           40200     
_________________________________________________________________
dense_22 (Dense)             (None, 16, 37)            7437      
Total params: 238,037
Trainable params: 238,037
Non-trainable params: 0
_________________________________________________________________
train model...
Epoch 1/10
256/256 [==============================] - ETA: 529s - loss: 3.644 - ETA: 273s - los

256/256 [==============================] - ETA: 9s - loss: 0.223 - ETA: 10s - loss: 0.30 - ETA: 10s - loss: 0.32 - ETA: 10s - loss: 0.28 - ETA: 11s - loss: 0.32 - ETA: 11s - loss: 0.31 - ETA: 11s - loss: 0.30 - ETA: 11s - loss: 0.28 - ETA: 11s - loss: 0.30 - ETA: 11s - loss: 0.29 - ETA: 11s - loss: 0.31 - ETA: 11s - loss: 0.37 - ETA: 11s - loss: 0.40 - ETA: 11s - loss: 0.41 - ETA: 11s - loss: 0.42 - ETA: 11s - loss: 0.41 - ETA: 11s - loss: 0.40 - ETA: 11s - loss: 0.41 - ETA: 11s - loss: 0.42 - ETA: 11s - loss: 0.41 - ETA: 11s - loss: 0.40 - ETA: 10s - loss: 0.40 - ETA: 10s - loss: 0.39 - ETA: 10s - loss: 0.38 - ETA: 10s - loss: 0.38 - ETA: 10s - loss: 0.38 - ETA: 10s - loss: 0.37 - ETA: 10s - loss: 0.37 - ETA: 10s - loss: 0.37 - ETA: 10s - loss: 0.36 - ETA: 10s - loss: 0.36 - ETA: 10s - loss: 0.36 - ETA: 10s - loss: 0.36 - ETA: 10s - loss: 0.36 - ETA: 10s - loss: 0.36 - ETA: 10s - loss: 0.35 - ETA: 10s - loss: 0.35 - ETA: 10s - loss: 0.35 - ETA: 10s - loss: 0.35 - ETA: 10s - loss: 0.35

256/256 [==============================] - ETA: 9s - loss: 0.047 - ETA: 11s - loss: 0.04 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.18 - ETA: 13s - loss: 0.16 - ETA: 13s - loss: 0.15 - ETA: 13s - loss: 0.13 - ETA: 13s - loss: 0.11 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.10 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10

...

256/256 [==============================] - ETA: 13s - loss: 4.6782e- - ETA: 15s - loss: 2.9216e- - ETA: 15s - loss: 3.1574e- - ETA: 15s - loss: 7.7482e- - ETA: 15s - loss: 9.8910e- - ETA: 15s - loss: 0.0020   - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00

256/256 [==============================] - ETA: 18s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 9.9950e- - ETA: 15s - loss: 0.0018   - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 15s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 13s - lo

256/256 [==============================] - ETA: 12s - loss: 0.00 - ETA: 13s - loss: 0.00 - ETA: 14s - loss: 9.4204e- - ETA: 14s - loss: 7.4297e- - ETA: 14s - loss: 8.1225e- - ETA: 15s - loss: 7.0752e- - ETA: 15s - loss: 6.1223e- - ETA: 15s - loss: 6.2350e- - ETA: 15s - loss: 6.4049e- - ETA: 15s - loss: 6.3922e- - ETA: 15s - loss: 8.8579e- - ETA: 15s - loss: 8.5523e- - ETA: 14s - loss: 8.1029e- - ETA: 14s - loss: 9.5906e- - ETA: 14s - loss: 8.9785e- - ETA: 14s - loss: 9.0195e- - ETA: 14s - loss: 8.8997e- - ETA: 14s - loss: 9.7439e- - ETA: 14s - loss: 9.9120e- - ETA: 14s - loss: 9.6360e- - ETA: 14s - loss: 0.0010   - ETA: 14s - loss: 0.00 - ETA: 14s - loss: 9.7853e- - ETA: 14s - loss: 9.5571e- - ETA: 14s - loss: 9.5450e- - ETA: 14s - loss: 9.3560e- - ETA: 14s - loss: 9.5105e- - ETA: 14s - loss: 9.2021e- - ETA: 13s - loss: 9.0825e- - ETA: 13s - loss: 9.7485e- - ETA: 13s - loss: 9.7757e- - ETA: 13s - loss: 9.5183e- - ETA: 13s - loss: 9.6534e- - ETA: 13s - loss: 9.4178e- - ETA: 13s - loss: 

256/256 [==============================] - ETA: 13s - loss: 0.00 - ETA: 13s - loss: 8.1479e- - ETA: 14s - loss: 6.0201e- - ETA: 14s - loss: 6.2917e- - ETA: 15s - loss: 5.6592e- - ETA: 15s - loss: 5.8143e- - ETA: 15s - loss: 5.0585e- - ETA: 15s - loss: 4.7785e- - ETA: 15s - loss: 6.4768e- - ETA: 15s - loss: 6.7177e- - ETA: 15s - loss: 6.1572e- - ETA: 15s - loss: 6.0760e- - ETA: 15s - loss: 5.6738e- - ETA: 15s - loss: 5.3927e- - ETA: 15s - loss: 5.4977e- - ETA: 15s - loss: 5.5179e- - ETA: 15s - loss: 5.3015e- - ETA: 14s - loss: 5.6342e- - ETA: 14s - loss: 5.4983e- - ETA: 14s - loss: 5.2386e- - ETA: 14s - loss: 5.1826e- - ETA: 14s - loss: 5.0273e- - ETA: 14s - loss: 4.9852e- - ETA: 14s - loss: 5.7191e- - ETA: 14s - loss: 6.2729e- - ETA: 14s - loss: 6.3334e- - ETA: 14s - loss: 6.2993e- - ETA: 14s - loss: 6.1181e- - ETA: 14s - loss: 5.9407e- - ETA: 14s - loss: 5.8558e- - ETA: 14s - loss: 5.7618e- - ETA: 14s - loss: 5.6697e- - ETA: 14s - loss: 5.5226e- - ETA: 14s - loss: 5.4985e- - ETA: 13s 

  frozenset({48}) frozenset({131}) frozenset({50}) frozenset({52})]]
File 10-epochs_3-samples.mid written


In [ ]:
n = [0,1,2,3,4,5,6,7,8]
n = np.array(n)
g = np.zeros((len(n),),dtype=np.int32)
print(g)
a = [0,5,7]
for i in a:
    print(np.array(i == np.arange(0, len(n) )))

In [86]:
def inner_pro(x1,x2,y1,y2):
    return (x1*y1)+(x2*y2)
x = [1.4, 1.6]
A1 = [1.5, 2.0, 1.6, 1.2, 1.5]
A2 = [1.7, 1.9, 1.8, 1.5, 1.0]
ans = []
for i in range(5):
    ans.append(inner_pro(x[0],x[1],A1[i],A2[i]) / (inner_pro(x[0],x[1],x[0],x[1]) * inner_pro(A1[i],A2[i],A1[i],A2[i])))
ans

[0.2074653076684687,
 0.16978126126545184,
 0.19530057979859625,
 0.2446219152457011,
 0.2518720217835262]

In [68]:
f = frozenset([1,2,3,4,5])
for i in f:
    print(i)

5
